In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import re
from scipy import signal
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
data1 = pd.read_csv('data_scaled.csv')

In [6]:
data1.head()

,x,y,z,activity
0,0.457376,0.689453,0.421540,Walking
1,0.438482,0.717446,0.420169,Walking
2,0.544191,0.755916,0.477840,Walking
3,0.487291,0.800191,0.390204,Walking
4,0.391350,0.836838,0.224368,Walking


In [7]:
X = data1[['x', 'y', 'z']]
y = data1['activity']

In [8]:
list=data1.activity.unique()
list

array(['Walking', 'Jogging', 'Stairs', 'Sitting', 'Standing', 'Typing',
       'Brushing Teeth', 'Eating Soup', 'Eating Chips', 'Eating Pasta',
       'Drinking from Cup', 'Eating Sandwich', 'Kicking (Soccer Ball)',
       'Playing Catch w/Tennis Ball', 'Dribbling (Basketball)', 'Writing',
       'Clapping', 'Folding Clothes'], dtype=object)

# Podział na okna czasowe

In [9]:
# # Przygotowanie cech rolling
# rolling_features = ['x_mean_rolling', 'y_mean_rolling', 'z_mean_rolling',
#                     'x_std_rolling', 'y_std_rolling', 'z_std_rolling',
#                     'x_range_rolling', 'y_range_rolling', 'z_range_rolling']
#
# # Ekstrakcja segmentów dla danych rolling
# rolling_segments = []
# for i in range(0, len(data) - N_TIME_STEPS, step):
#     segment = data[rolling_features].iloc[i: i + N_TIME_STEPS].values.flatten()
#     rolling_segments.append(segment)
#
# rolling_segments = np.asarray(rolling_segments)


In [10]:
# import numpy as np
# import pandas as pd
# from scipy.stats import mode
# from sklearn.model_selection import train_test_split
# import tensorflow as tf
#
# # Constants
# N_TIME_STEPS = 100
# N_FEATURES = 3
# step = 30
# RANDOM_SEED = 42
#
# segments = []
# labels = []
# from collections import Counter
# # Loop to segment the data
# for i in range(0, len(data1) - N_TIME_STEPS, step):
#     xs = data1['x'].values[i: i + N_TIME_STEPS]
#     ys = data1['y'].values[i: i + N_TIME_STEPS]
#     zs = data1['z'].values[i: i + N_TIME_STEPS]
#
#     # Replace stats.mode with an alternative using collections.Counter
#     segment_labels = data1['activity'][i: i + N_TIME_STEPS]
#     label = Counter(segment_labels).most_common(1)[0][0]  # Calculate mode
#
#     # Append data and labels
#     segments.append([xs, ys, zs])
#     labels.append(label)
#
# # Reshape segments and encode labels
# reshaped_segments = np.asarray(segments, dtype=np.float32).reshape(-1, N_TIME_STEPS, N_FEATURES)
# labels = np.asarray(pd.get_dummies(labels), dtype=np.float32)
#
# # # Split the data
# # X_train, X_test, y_train, y_test = train_test_split(
# #     reshaped_segments, labels, test_size=0.2, random_state=RANDOM_SEED)
#
# from sklearn.model_selection import train_test_split
#
# # Podział na dane treningowe i tymczasowe (walidacyjne + testowe)
# X_train, X_temp, y_train, y_temp = train_test_split(reshaped_segments, labels, test_size=0.3, random_state=42)
#
# # Podział tymczasowych danych na walidacyjne i testowe
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
#
# # Sprawdzenie rozmiarów
# print(f"Trening: {X_train.shape}, Walidacja: {X_val.shape}, Testy: {X_test.shape}")
#
#
# # Expand dimensions for TensorFlow compatibility
# X_train = tf.expand_dims(X_train, axis=-1)
# X_test = tf.expand_dims(X_test, axis=-1)
# X_val = tf.expand_dims(X_val, axis=-1)

In [16]:
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Constants
N_TIME_STEPS = 100
N_FEATURES = 3  # Początkowo 3 cechy: x, y, z
step = 30
RANDOM_SEED = 42

segments = []
labels = []

# Loop to segment the data
for i in range(0, len(data1) - N_TIME_STEPS, step):
    xs = data1['x'].values[i: i + N_TIME_STEPS]
    ys = data1['y'].values[i: i + N_TIME_STEPS]
    zs = data1['z'].values[i: i + N_TIME_STEPS]

    # Średnia w czasie (dla każdej cechy w segmencie)
    mean_x = np.mean(xs)
    mean_y = np.mean(ys)
    mean_z = np.mean(zs)

    # Tworzenie segmentu
    segment = np.column_stack((xs, ys, zs))  # Oryginalne dane czasowe

    # Dodanie średnich jako dodatkowych cech (po czasie)
    extra_features = [mean_x, mean_y, mean_z]
    segment = np.hstack((segment, np.tile(extra_features, (N_TIME_STEPS, 1))))  # Powielanie cech na każdym kroku czasowym

    # Najczęściej występująca etykieta
    segment_labels = data1['activity'][i: i + N_TIME_STEPS]
    label = Counter(segment_labels).most_common(1)[0][0]

    # Append segment and label
    segments.append(segment)
    labels.append(label)

# Przekształć dane w odpowiedni format
reshaped_segments = np.asarray(segments, dtype=np.float32)  # (n_samples, N_TIME_STEPS, N_FEATURES + extra_features)
labels = np.asarray(pd.get_dummies(labels), dtype=np.float32)

# Podział na dane treningowe i tymczasowe (walidacyjne + testowe)
X_train, X_temp, y_train, y_temp = train_test_split(reshaped_segments, labels, test_size=0.3, random_state=RANDOM_SEED)

# Podział tymczasowych danych na walidacyjne i testowe
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=RANDOM_SEED)

# Sprawdzenie rozmiarów
print(f"Trening: {X_train.shape}, Walidacja: {X_val.shape}, Testy: {X_test.shape}")

# Expand dimensions for TensorFlow compatibility (jeśli wymagane przez model)
X_train = tf.expand_dims(X_train, axis=-1)
X_test = tf.expand_dims(X_test, axis=-1)
X_val = tf.expand_dims(X_val, axis=-1)


Trening: (15129, 100, 6), Walidacja: (3242, 100, 6), Testy: (3242, 100, 6)


In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout


from tensorflow.keras.layers import BatchNormalization

model = Sequential([
    Input(shape=(X_train.shape[1], X_train.shape[2])),

    Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=5, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=7, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=256, kernel_size=7, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),

    Flatten(),

    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(18, activation='softmax')
])

# Definicja modelu
# model = Sequential([
#     Input(shape=(X_train.shape[1], X_train.shape[2])),
#     # Pierwsza warstwa konwolucyjna
#     Conv1D(filters=32, kernel_size=5, activation='relu'),
#     MaxPooling1D(pool_size=2),
#
#     # Druga warstwa konwolucyjna
#     Conv1D(filters=64, kernel_size=5, activation='relu'),
#     MaxPooling1D(pool_size=2),
#
#     # Trzecia warstwa konwolucyjna
#     Conv1D(filters=128, kernel_size=5, activation='relu'),
#     MaxPooling1D(pool_size=2),
#
#     # Spłaszczenie danych
#     Flatten(),
#
#     # Warstwy Dense
#     Dense(128, activation='relu'),
#     Dropout(0.5),  # Zapobieganie przeuczeniu
#     Dense(64, activation='relu'),
#
#     # Warstwa wyjściowa
#     Dense(18, activation='softmax')
# ])
# model = Sequential([
#     Input(shape=(X_train.shape[1], X_train.shape[2])),
#     # Pierwsza warstwa konwolucyjna
#     Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'),
#     MaxPooling1D(pool_size=2),
#
#     # Druga warstwa konwolucyjna
#     Conv1D(filters=64, kernel_size=5, activation='relu', padding='same'),
#     MaxPooling1D(pool_size=2),
#
#     # Trzecia warstwa konwolucyjna
#     Conv1D(filters=128, kernel_size=5, activation='relu', padding='same'),
#     MaxPooling1D(pool_size=2),
#
#     # Spłaszczenie danych
#     Flatten(),
#
#     # Warstwy Dense
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(64, activation='relu'),
#
#     # Warstwa wyjściowa
#     Dense(18, activation='softmax')
# ])


In [18]:

# Kompilacja modelu
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
mc = ModelCheckpoint('cnn_best_model3.keras', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                    epochs=100, batch_size=32, verbose=2, callbacks=[es, mc])


Epoch 1/100

Epoch 1: val_accuracy improved from -inf to 0.25632, saving model to cnn_best_model3.keras
473/473 - 18s - 39ms/step - accuracy: 0.3154 - loss: 1.9893 - val_accuracy: 0.2563 - val_loss: 2.3735
Epoch 2/100


In [189]:
from tensorflow.keras.models import load_model

# Wczytaj najlepszy model
best_model = load_model('cnn_best_model.keras')


In [190]:
# Ewaluacja modelu
loss, accuracy = best_model.evaluate(X_test, y_test, verbose=1)

print(f"Dokładność na zbiorze testowym: {accuracy:.2f}")


102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8971 - loss: 0.3097
Dokładność na zbiorze testowym: 0.90
